In [ ]:
! nvidia-smi

Sat May 10 07:42:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install accelerate
!pip install huggingface_hub

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
! pip install rich

## model name
in this notebook will use ***Qwen/Qwen2.5-Coder-1.5B-Instruct*** cause small usage model

In [ ]:
model1 = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
model2 = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
model3 = 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B'
model4 = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B'
model5 = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-32B'
model6 = 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'
model7 = 'Qwen/QwQ-32B'
model8 = 'Qwen/Qwen2.5-Coder-1.5B-Instruct'
model9 = 'Qwen/Qwen2.5-7B-Instruct'
model10 = 'Qwen/Qwen2.5-14B-Instruct'
model11 = "distilgpt2"
model12 = "facebook/opt-1.3b"

In [ ]:
from IPython.display import Markdown, display

## Fucntions
- `load_model` use for get model and tokenize from transformer
- `get_size` to check size of model
- `use_gpu` to use cuda GPU instead run on CPU if available
- `gen_chat_pipe1` use model and tokenize from `load_model`
- `pipe_load` easier way to load model without pre-load model
- `gen_text_pipe` use pipe from `pipe_load`
- `gen_text_model` use model and tokenize from pre-load from `load_model`


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def load_model(model_name):
  model_name = model_name  # or other model
  token = "xxxxxxxxxxxxxxxxxxxxxxxxx"  # Replace with your actual token

  try:
      model = AutoModelForCausalLM.from_pretrained(model_name, token=token)
      tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
      print("Model and tokenizer loaded successfully!")

      return model, tokenizer

  except OSError as e:
      print(f"Error: {e}")
      print("Please verify the model name and ensure you have the correct token.")


# Get the size of the model in memory
def get_size(model):
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()

  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()

  size_all_mb = (param_size + buffer_size) / 1024**2
  print(f"Model size: {size_all_mb:.2f} MB")

# use GPU cuda
def use_gpu(model):
  if torch.cuda.is_available():
      device = 0
      print("GPU available")
  else:
      device = -1
      print("No GPU available")

from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

def gen_chat_pipe1(model, tokenizer, content:str, device, max_new_tokens=1024):

  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
  # pipe = pipeline("chat-generation", model=model, tokenizer=tokenizer)


  message  = [
      {'role':"user",'content':content}
  ]

  result = pipe(message, max_new_tokens=max_new_tokens)

  return result

def pipe_load(model_name, device, max_length=5000):
  # Create the pipeline
  pipe = pipeline("text-generation", model=model_name, max_length=max_length, num_return_sequences=1, device=device)

  return pipe

def gen_text_pipe(pipe, message:str, truncation=True, max_new_tokens=1024):
  # Generate the result
  result = pipe(message, truncation=truncation, max_new_tokens=1024)

  return pipe, result[0]

def gen_text_model(model, tokenizer, content: str, max_length=1000):

    device = model.device  # Get the model's device (CPU or CUDA)

    inputs = tokenizer(content, return_tensors="pt").to(device)  # Move inputs to same device as model
    output_ids = model.generate(**inputs, max_length=max_length)

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    # Remove the question if it appears at the beginning
    if response.startswith(content):
        response = response[len(content):].strip()

    # After removing question, if the first character is weird (like "?" or newline), remove it
    response = response.lstrip(" .?\n")

    # # (Optional) If there's any "Assistant:" marker, clean it too
    # if "Assistant:" in response:
    #     response = response.split("Assistant:", 1)[-1].strip()

    return output_ids , response

## Usage case
**Description**: using Prompt-based Generation with *Qwen/Qwen2.5-Coder-1.5B-Instruct* model

In [ ]:
model, tokenizer = load_model(model8)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [ ]:
use_gpu(model)

GPU available


In [ ]:
model8

'Qwen/Qwen2.5-Coder-1.5B-Instruct'

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [ ]:
get_size(model)

Model size: 5888.80 MB




---



### Example 1
use function `gen_chat_pipe1()` and use laoded model from `load_model`

In [ ]:
answer1 = gen_chat_pipe1(model, tokenizer, "How to sleep well ?")

Device set to use cuda:0


In [ ]:
answer1

[{'generated_text': [{'role': 'user', 'content': 'How to sleep well ?'},
   {'role': 'assistant',
    'content': "Sleeping well is crucial for maintaining good health and productivity. Here are some tips that can help you get a better night's rest:\n\n1. Establish a regular sleep schedule: Try to go to bed and wake up at the same time every day, even on weekends.\n\n2. Create a relaxing bedtime routine: Engage in activities that promote relaxation, such as reading, meditation, or taking a warm bath.\n\n3. Avoid caffeine and alcohol before bedtime: These can interfere with your ability to fall asleep.\n\n4. Limit screen time before bedtime: The blue light from electronic devices can suppress melatonin production, making it difficult to fall asleep.\n\n5. Create a comfortable sleeping environment: Make sure your bedroom is dark, quiet, and cool. Use blackout curtains or eye masks if necessary.\n\n6. Exercise regularly: Regular physical activity can help improve your sleep quality, but av

In [ ]:
answer1[0]['generated_text'][1]['content']

"Sleeping well is crucial for maintaining good health and productivity. Here are some tips that can help you get a better night's rest:\n\n1. Establish a regular sleep schedule: Try to go to bed and wake up at the same time every day, even on weekends.\n\n2. Create a relaxing bedtime routine: Engage in activities that promote relaxation, such as reading, meditation, or taking a warm bath.\n\n3. Avoid caffeine and alcohol before bedtime: These can interfere with your ability to fall asleep.\n\n4. Limit screen time before bedtime: The blue light from electronic devices can suppress melatonin production, making it difficult to fall asleep.\n\n5. Create a comfortable sleeping environment: Make sure your bedroom is dark, quiet, and cool. Use blackout curtains or eye masks if necessary.\n\n6. Exercise regularly: Regular physical activity can help improve your sleep quality, but avoid exercising too close to bedtime.\n\n7. Manage stress: Stress can affect your ability to fall asleep. Try tech

In [ ]:
assistant_msg = next(
    (msg["content"] for msg in answer1[0]["generated_text"] if msg.get("role") == "assistant"),
    None
)


In [ ]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()

if assistant_msg:
    console.print(Markdown(assistant_msg))


Sleeping well is crucial for maintaining good health and productivity. Here are some tips that can help you get a  
better night's rest:                                                                                               

 1 Establish a regular sleep schedule: Try to go to bed and wake up at the same time every day, even on weekends.  
 2 Create a relaxing bedtime routine: Engage in activities that promote relaxation, such as reading, meditation, or
   taking a warm bath.                                                                                             
 3 Avoid caffeine and alcohol before bedtime: These can interfere with your ability to fall asleep.                
 4 Limit screen time before bedtime: The blue light from electronic devices can suppress melatonin production,     
   making it difficult to fall asleep.                                                                             
 5 Create a comfortable sleeping environment: Make sure your bedroom is dark, quiet, and cool. Use blackout        
   curtains or eye masks if necessary.                                                                             
 6 Exercise regularly: Regular physical activity can help improve your sleep quality, but avoid exercising too     
   close to bedtime.                                                                                               
 7 Manage stress: Stress can affect your ability to fall asleep. Try techniques such as deep breathing, progressive
   muscle relaxation, or yoga to manage stress.                                                                    

Remember, everyone has different sleep needs, so it may take some experimentation to find what works best for you.



---



### Example 2
use function `pipe_load` and `gen_text_pipe`

In [ ]:
pipe = pipe_load(model8)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
answer2 = gen_text_pipe(pipe, "How to sleep well ?")

In [ ]:
answer2

(<transformers.pipelines.text_generation.TextGenerationPipeline at 0x7fc2b2eef790>,
 {'generated_text': "How to sleep well ? How do you feel when you go to bed at night?\n\nTo sleep well, it's important to establish a regular bedtime routine and create an environment that is conducive to relaxation. Here are some tips:\n\n1. Establish a regular bedtime routine: This can include activities such as reading, meditation, or exercise.\n\n2. Create a comfortable sleeping environment: Make sure your bedroom is dark, quiet, and cool. Use comfortable bedding and pillows.\n\n3. Avoid caffeine and alcohol before bedtime: These can interfere with sleep quality.\n\n4. Limit screen time before bedtime: The blue light from screens can suppress melatonin production, making it harder to fall asleep.\n\n5. Practice relaxation techniques: Activities such as deep breathing, progressive muscle relaxation, or yoga can help reduce stress and improve sleep quality.\n\nWhen you go to bed at night, try to relax

In [ ]:
answer2[1]['generated_text']

"How to sleep well ? How do you feel when you go to bed at night?\n\nTo sleep well, it's important to establish a regular bedtime routine and create an environment that is conducive to relaxation. Here are some tips:\n\n1. Establish a regular bedtime routine: This can include activities such as reading, meditation, or exercise.\n\n2. Create a comfortable sleeping environment: Make sure your bedroom is dark, quiet, and cool. Use comfortable bedding and pillows.\n\n3. Avoid caffeine and alcohol before bedtime: These can interfere with sleep quality.\n\n4. Limit screen time before bedtime: The blue light from screens can suppress melatonin production, making it harder to fall asleep.\n\n5. Practice relaxation techniques: Activities such as deep breathing, progressive muscle relaxation, or yoga can help reduce stress and improve sleep quality.\n\nWhen you go to bed at night, try to relax and prepare yourself for sleep. If you find it difficult to fall asleep, try to stay in bed for the sam

In [ ]:
display(Markdown(answer2[1]['generated_text']))

How to sleep well ? How do you feel when you go to bed at night?

To sleep well, it's important to establish a regular bedtime routine and create an environment that is conducive to relaxation. Here are some tips:

1. Establish a regular bedtime routine: This can include activities such as reading, meditation, or exercise.

2. Create a comfortable sleeping environment: Make sure your bedroom is dark, quiet, and cool. Use comfortable bedding and pillows.

3. Avoid caffeine and alcohol before bedtime: These can interfere with sleep quality.

4. Limit screen time before bedtime: The blue light from screens can suppress melatonin production, making it harder to fall asleep.

5. Practice relaxation techniques: Activities such as deep breathing, progressive muscle relaxation, or yoga can help reduce stress and improve sleep quality.

When you go to bed at night, try to relax and prepare yourself for sleep. If you find it difficult to fall asleep, try to stay in bed for the same amount of time each day and avoid napping during the day. If your sleep problems persist, consider consulting a healthcare professional.



---



### Example 3
use function `gen_text_model` from model that loaded from `load_model`

In [ ]:
output_ids , answer3 = gen_text_model(model, tokenizer, "How to sleep well ?")

In [ ]:
answer3

"How do you feel when you are sleeping ?\n\nSleep is an important aspect of our lives. It helps us rest and recharge, which is essential for maintaining good health. However, not everyone gets enough quality sleep. Here are some tips on how to get better sleep:\n\n1. Establish a regular sleep schedule: Try to go to bed and wake up at the same time every day, even on weekends. This can help regulate your body's internal clock.\n\n2. Create a relaxing bedtime routine: Before going to bed, try to do something that relaxes you, such as reading a book, listening to music, or taking a warm bath.\n\n3. Limit exposure to light: Exposure to bright lights before bedtime can interfere with your ability to fall asleep. Try to keep your bedroom dark and quiet.\n\n4. Avoid caffeine and alcohol: These substances can interfere with your ability to fall asleep. Try to avoid them in the hours leading up to bedtime.\n\n5. Exercise regularly: Regular exercise can help improve your sleep quality. Aim for a

In [ ]:
display(Markdown(answer3))

How do you feel when you are sleeping ?

Sleep is an important aspect of our lives. It helps us rest and recharge, which is essential for maintaining good health. However, not everyone gets enough quality sleep. Here are some tips on how to get better sleep:

1. Establish a regular sleep schedule: Try to go to bed and wake up at the same time every day, even on weekends. This can help regulate your body's internal clock.

2. Create a relaxing bedtime routine: Before going to bed, try to do something that relaxes you, such as reading a book, listening to music, or taking a warm bath.

3. Limit exposure to light: Exposure to bright lights before bedtime can interfere with your ability to fall asleep. Try to keep your bedroom dark and quiet.

4. Avoid caffeine and alcohol: These substances can interfere with your ability to fall asleep. Try to avoid them in the hours leading up to bedtime.

5. Exercise regularly: Regular exercise can help improve your sleep quality. Aim for at least 30 minutes of moderate activity most days of the week.

6. Manage stress: Stress can interfere with your ability to fall asleep. Try to manage your stress through activities such as meditation, yoga, or deep breathing exercises.

When you are sleeping, it is important to feel relaxed and comfortable. You should be able to move around freely without feeling restricted or uncomfortable. If you find yourself having trouble falling asleep or staying asleep, it may be helpful to consult with a healthcare professional. They can provide you with additional advice and treatment options.



---

